In [1]:
"""
运行前请移除results文件夹中的所有数据结果
"""

import os
import Simulator
import pandas as pd
import glob
import time

from iBridgeFunctions import *
from cobra.io import read_sbml_model

import re

In [2]:
filename = r"C:\Users\Victor\PycharmProjects\pythonProject\iBridge\input\ecYeastGEM_batch.xml"
#filename=r"C:\Users\Victor\PycharmProjects\pythonProject\iBridge\input\iJO1366.xml"
biomass_reaction = 'r_2111'
target_reaction = 'r_2024'
#target_reaction = 'r_2051'
#target_reaction = 'r_1589'
#biomass_reaction='BIOMASS_Ec_iJO1366_core_53p95M'
#target_reaction='EX_ptrc_e'

input_dir = './input'
output_dir = './results'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

constrict={}

In [3]:
#model = read_sbml_model(filename)

In [4]:
#model.reactions.get_by_id('r_0695').genes

In [5]:
#list(model.reactions.get_by_id('r_0695').genes)

In [6]:
#model.reactions.get_by_id('r_2213').genes

In [7]:
#list(model.reactions.get_by_id('r_2213').genes)

In [8]:
# ans = []
# ans.append(model.reactions.get_by_id('r_0695').genes)
# ans.append(model.reactions.get_by_id('r_2213').genes)
# ans

In [9]:
#pd.DataFrame(data=ans,columns=['OE'])

In [10]:
simulator = Simulator.Simulator()
model_metabolites, model_reactions, Smatrix, lower_boundary_constraints, upper_boundary_constraints, objective_reaction = simulator.read_model(filename)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-17


In [11]:
corr_output = '%s/corr_%s.csv'%(output_dir,target_reaction)
cov_output = '%s/cov_%s.csv'%(output_dir, target_reaction)

In [12]:
flux_distribution_dict = generate_multiple_flux_profiles(simulator, biomass_reaction, target_reaction, constrict)

Count: 1	Biomass flux: 0.376828
Count: 2	Biomass flux: 0.337051
Count: 3	Biomass flux: 0.297275
Count: 4	Biomass flux: 0.257499
Count: 5	Biomass flux: 0.217723
Count: 6	Biomass flux: 0.177946
Count: 7	Biomass flux: 0.138170
Count: 8	Biomass flux: 0.098481
Count: 9	Biomass flux: 0.058669
Count: 10	Biomass flux: 0.018858


In [13]:
flux_df = pd.DataFrame.from_dict(flux_distribution_dict)
flux_corr_df = flux_df.abs().T.corr()
flux_cov_df = flux_df.abs().T.cov()

In [14]:
flux_corr_df = flux_corr_df[target_reaction].loc[simulator.model_reactions]
flux_corr_df.to_csv(corr_output,header=0)
flux_cov_df = flux_cov_df[target_reaction].loc[simulator.model_reactions]
flux_cov_df.to_csv(cov_output,header=0)

write_header(corr_output, target_reaction)
write_header(cov_output, target_reaction)

In [15]:
df = pd.read_csv(cov_output, index_col=0)

In [16]:
#ecModel = read_sbml_model(r"C:\Users\Victor\PycharmProjects\pythonProject\iBridge\input\iJO1366.xml")
ecModel = read_sbml_model(r"C:\Users\Victor\PycharmProjects\pythonProject\iBridge\input\ecYeastGEM_batch.xml")
met_info = metabolite_set(ecModel)

In [17]:
final_dic = {}
fluxsum_dic = calculate_MetScore_sum(ecModel,  dict(df[target_reaction]))
final_dic[target_reaction]=fluxsum_dic

In [18]:
metscore_df = pd.DataFrame.from_dict(final_dic)
metscore_df.to_csv('%s/MetScore_%s.csv'%(output_dir, target_reaction))

In [19]:
flux_corr_df = pd.read_csv(corr_output, index_col=0)
flux_cov_df = pd.read_csv(cov_output, index_col=0)

In [20]:
output_file = '%s/Final_MetScore_%s.txt'%(output_dir, target_reaction)

In [21]:
select_candidates(ecModel, target_reaction, output_file, metscore_df, flux_corr_df, flux_cov_df)

Number of positive candidate reactions: 52
Number of negative candidate reactions: 91


In [22]:
df2 = pd.read_table(os.path.abspath(output_file))

In [23]:
files = glob.glob('%s/Final_*.txt'%output_dir)
for filename in files:
    basename = os.path.basename(filename)
    df2 = pd.read_table(filename)
    make_candidate_reaction_sets(df2, basename, input_dir, output_dir)

In [24]:
files = glob.glob('%s/application_results/*.txt'%output_dir)

for each_file in files:
    s = time.time()
    basename = os.path.basename(each_file)
    target_reaction_id = basename.split('Final_MetScore_')[1].strip().replace('.txt', '')

    corr_file = glob.glob('%s/corr*%s*.csv'%(output_dir, target_reaction_id))[0]
    cov_file = glob.glob('%s/cov*%s*.csv'%(output_dir, target_reaction_id))[0]

    flux_corr_df = pd.read_csv(corr_file, index_col=0)
    flux_cov_df = pd.read_csv(cov_file, index_col=0)

    print('Target reaction: %s'%(target_reaction_id))

    up_reaction_list = check_up_reaction(each_file, met_info, flux_corr_df, flux_cov_df, output_dir)
    down_reaction_list = check_down_reaction(each_file, met_info, flux_corr_df, flux_cov_df, output_dir)
    
    e = time.time()
    print('Elapsed time: %fs'%(e-s))

Target reaction: r_2024
Elapsed time: 500.841484s


In [25]:
up_reaction_list

['r_0018No1',
 'r_0018No2',
 'r_0018No3',
 'r_0018_REVNo1',
 'r_0018_REVNo2',
 'r_0018_REVNo3',
 'r_0026No1',
 'r_0026No2',
 'r_0026No3',
 'r_0026No4',
 'r_0073_REVNo1',
 'r_0073_REVNo2',
 'r_0080No1',
 'r_0080No2',
 'r_0092_REVNo1',
 'r_0092_REVNo2',
 'r_0164No1',
 'r_0164No2',
 'r_0168No1',
 'r_0168No2',
 'r_0168No3',
 'r_0171No1',
 'r_0171No2',
 'r_0181No1',
 'r_0181No2',
 'r_0184No1',
 'r_0184No2',
 'r_0199No1',
 'r_0199No2',
 'r_0211No1',
 'r_0211No2',
 'r_0306No1',
 'r_0306No2',
 'r_0308No1',
 'r_0308No2',
 'r_0366No1',
 'r_0366No2',
 'r_0366No3',
 'r_0366No4',
 'r_0366No5',
 'r_0475No1',
 'r_0475No2',
 'r_0475No1',
 'r_0475No2',
 'r_0477No1',
 'r_0477No2',
 'r_0481No1',
 'r_0481No2',
 'r_0481No3',
 'r_0481No4',
 'r_0481No1',
 'r_0481No2',
 'r_0481No3',
 'r_0481No4',
 'r_0483No1',
 'r_0483No2',
 'r_0483No3',
 'r_0483No4',
 'r_0483No5',
 'r_0483No6',
 'r_0483_REVNo1',
 'r_0483_REVNo2',
 'r_0483_REVNo3',
 'r_0483_REVNo4',
 'r_0483_REVNo5',
 'r_0483_REVNo6',
 'r_0550No1',
 'r_0550No

In [26]:
len(up_reaction_list)

1073

In [27]:
down_reaction_list

['arm_r_0018',
 'arm_r_0018_REV',
 'arm_r_0024',
 'arm_r_0026',
 'arm_r_0033',
 'arm_r_0042',
 'arm_r_0073',
 'arm_r_0076',
 'arm_r_0078',
 'arm_r_0080',
 'arm_r_0091',
 'arm_r_0092',
 'arm_r_0164',
 'arm_r_0168',
 'arm_r_0171',
 'arm_r_0172',
 'arm_r_0181',
 'arm_r_0184',
 'arm_r_0185',
 'arm_r_0198',
 'arm_r_0199',
 'arm_r_0211',
 'arm_r_0211',
 'arm_r_0227',
 'arm_r_0271',
 'arm_r_0306',
 'arm_r_0306',
 'arm_r_0307',
 'arm_r_0308',
 'arm_r_0364',
 'arm_r_0366_REV',
 'arm_r_0448',
 'arm_r_0475',
 'arm_r_0475',
 'arm_r_0477',
 'arm_r_0481',
 'arm_r_0483',
 'arm_r_0483_REV',
 'arm_r_0489',
 'arm_r_0510',
 'arm_r_0558',
 'arm_r_0565',
 'arm_r_0570_REV',
 'arm_r_0688',
 'arm_r_0688',
 'arm_r_0716',
 'arm_r_0726',
 'arm_r_0757',
 'arm_r_0804',
 'arm_r_0889',
 'arm_r_1031',
 'arm_r_1038',
 'arm_r_1051',
 'arm_r_1095_REV',
 'arm_r_1168',
 'arm_r_2116',
 'arm_r_3120',
 'arm_r_3121',
 'arm_r_3122',
 'arm_r_3123',
 'arm_r_3124',
 'arm_r_3125',
 'arm_r_3126',
 'arm_r_3127',
 'arm_r_3152',
 'arm

In [28]:
len(down_reaction_list)

919

In [29]:
up_reaction_filter = [up_reaction for up_reaction in up_reaction_list if not up_reaction.startswith('arm') and not up_reaction.startswith('pmet')]
up_reaction_filter

['r_0018No1',
 'r_0018No2',
 'r_0018No3',
 'r_0018_REVNo1',
 'r_0018_REVNo2',
 'r_0018_REVNo3',
 'r_0026No1',
 'r_0026No2',
 'r_0026No3',
 'r_0026No4',
 'r_0073_REVNo1',
 'r_0073_REVNo2',
 'r_0080No1',
 'r_0080No2',
 'r_0092_REVNo1',
 'r_0092_REVNo2',
 'r_0164No1',
 'r_0164No2',
 'r_0168No1',
 'r_0168No2',
 'r_0168No3',
 'r_0171No1',
 'r_0171No2',
 'r_0181No1',
 'r_0181No2',
 'r_0184No1',
 'r_0184No2',
 'r_0199No1',
 'r_0199No2',
 'r_0211No1',
 'r_0211No2',
 'r_0306No1',
 'r_0306No2',
 'r_0308No1',
 'r_0308No2',
 'r_0366No1',
 'r_0366No2',
 'r_0366No3',
 'r_0366No4',
 'r_0366No5',
 'r_0475No1',
 'r_0475No2',
 'r_0475No1',
 'r_0475No2',
 'r_0477No1',
 'r_0477No2',
 'r_0481No1',
 'r_0481No2',
 'r_0481No3',
 'r_0481No4',
 'r_0481No1',
 'r_0481No2',
 'r_0481No3',
 'r_0481No4',
 'r_0483No1',
 'r_0483No2',
 'r_0483No3',
 'r_0483No4',
 'r_0483No5',
 'r_0483No6',
 'r_0483_REVNo1',
 'r_0483_REVNo2',
 'r_0483_REVNo3',
 'r_0483_REVNo4',
 'r_0483_REVNo5',
 'r_0483_REVNo6',
 'r_0550No1',
 'r_0550No

In [30]:
down_reaction_filter = [down_reaction for down_reaction in down_reaction_list if not down_reaction.startswith('arm') and not down_reaction.startswith('pmet')]
down_reaction_filter

['r_0060_REVNo1',
 'r_0703No1',
 'r_0019No1',
 'r_0663No1',
 'r_0696No1',
 'r_0909No1',
 'r_0910No1',
 'r_0725No1',
 'r_1624No1',
 'r_0231No1',
 'r_1614',
 'r_1616',
 'r_0328No1',
 'r_1618',
 'r_0329No1',
 'r_0525No1',
 'r_0015No1',
 'r_0023No1',
 'r_1099',
 'r_4355_REV',
 'r_0310No1',
 'r_1063_REVNo1',
 'r_0939No1',
 'r_0929No1',
 'r_0936No1',
 'r_0937No1',
 'r_0559No1',
 'r_0671No1',
 'r_0045No1',
 'r_1087No1',
 'r_0918No1',
 'r_0317No1',
 'r_0066No1',
 'r_2065',
 'r_0331No1',
 'r_4292_REVNo1',
 'r_0681No1',
 'r_0069No1',
 'r_0699No1',
 'r_0013No1',
 'r_0236No1',
 'r_0241No1',
 'r_0079No1',
 'r_0079No1',
 'r_1001No1',
 'r_0732No1',
 'r_0732_REVNo1',
 'r_2065',
 'r_0014No1',
 'r_2030',
 'r_0088_REVNo1',
 'r_0089_REVNo1',
 'r_0915No1',
 'r_0915No1',
 'r_0466No1',
 'r_0188No1',
 'r_0348No1',
 'r_1744',
 'r_0524No1',
 'r_0526No1',
 'r_4397',
 'r_0173No1',
 'r_0369No1',
 'r_0656No1',
 'r_0657No1',
 'r_0173No1',
 'r_4574',
 'r_1613',
 'r_4376',
 'r_4495',
 'r_0144No1',
 'r_0155No1',
 'r_09

In [31]:
ecModel.reactions.get_by_id('r_4208No1').genes

frozenset({<Gene YKR076W at 0x1650b7af350>})

In [32]:
oe_genes = []
kd_genes = []

In [33]:
for up_reaction in up_reaction_filter:
    fs = ecModel.reactions.get_by_id(up_reaction).genes
    if len(fs)== 0:
        continue
    oe_genes.append(fs)
    
for down_reaction in down_reaction_filter:
    fs = ecModel.reactions.get_by_id(down_reaction).genes
    if len(fs)== 0:
        continue
    kd_genes.append(fs)

In [34]:
oe_genes

[frozenset({<Gene YER152C at 0x1650b7cc790>}),
 frozenset({<Gene YGL202W at 0x1650b7cf490>}),
 frozenset({<Gene YJL060W at 0x1650b7dff10>}),
 frozenset({<Gene YER152C at 0x1650b7cc790>}),
 frozenset({<Gene YGL202W at 0x1650b7cf490>}),
 frozenset({<Gene YJL060W at 0x1650b7dff10>}),
 frozenset({<Gene YHR208W at 0x1650b7dda10>}),
 frozenset({<Gene YJR148W at 0x1650b847050>}),
 frozenset({<Gene YGL202W at 0x1650b7cf490>}),
 frozenset({<Gene YHR137W at 0x1650b7dd310>}),
 frozenset({<Gene YLR410W at 0x1650b82b1d0>}),
 frozenset({<Gene YOR163W at 0x1650b82d790>}),
 frozenset({<Gene YGL125W at 0x1650b7cec90>}),
 frozenset({<Gene YPL023C at 0x1650b82f990>}),
 frozenset({<Gene YLR410W at 0x1650b82b1d0>}),
 frozenset({<Gene YOR163W at 0x1650b82d790>}),
 frozenset({<Gene YDR368W at 0x1650b7f8050>}),
 frozenset({<Gene YHR104W at 0x1650b7dcf10>}),
 frozenset({<Gene YCR105W at 0x1650b7aa8d0>}),
 frozenset({<Gene YDR368W at 0x1650b7f8050>}),
 frozenset({<Gene YMR318C at 0x1650b7e3910>}),
 frozenset({<

In [35]:
kd_genes

[frozenset({<Gene YGL009C at 0x1650b7cdf90>}),
 frozenset({<Gene YNL045W at 0x1650b7e3f10>}),
 frozenset({<Gene YHR063C at 0x1650b7dc510>}),
 frozenset({<Gene YJR148W at 0x1650b847050>}),
 frozenset({<Gene YOL151W at 0x1650b817f50>}),
 frozenset({<Gene YCL030C at 0x1650b7a92d0>}),
 frozenset({<Gene YCL030C at 0x1650b7a92d0>}),
 frozenset({<Gene YGR204W at 0x1650b7d5dd0>}),
 frozenset({<Gene YER183C at 0x1650b7ccc90>}),
 frozenset({<Gene YNL280C at 0x1650b815650>}),
 frozenset({<Gene YNL141W at 0x1650b814950>}),
 frozenset({<Gene YLR245C at 0x1650b8299d0>}),
 frozenset({<Gene YBL033C at 0x1650b80f010>}),
 frozenset({<Gene YBR153W at 0x1650b80c4d0>}),
 frozenset({<Gene YGL009C at 0x1650b7cdf90>}),
 frozenset({<Gene YOR222W at 0x1650b82e390>, <Gene YPL134C at 0x1650b7f4bd0>}),
 frozenset({<Gene YAL012W at 0x1650b7ebc50>}),
 frozenset({<Gene YGL202W at 0x1650b7cf490>}),
 frozenset({<Gene YBR166C at 0x1650b80c610>}),
 frozenset({<Gene YMR020W at 0x1650b7e1010>}),
 frozenset({<Gene YMR020W a

In [36]:
oe_genes_final = list(set(oe_genes))
oe_genes_final

[frozenset({<Gene YFL018C at 0x1650b7cd090>}),
 frozenset({<Gene YPR020W at 0x1650b7f63d0>}),
 frozenset({<Gene YOR202W at 0x1650b82e090>}),
 frozenset({<Gene YNL333W at 0x1650b7afe50>}),
 frozenset({<Gene YLR410W at 0x1650b82b1d0>}),
 frozenset({<Gene YMR251W at 0x1650b7af450>}),
 frozenset({<Gene YMR300C at 0x1650b7e3610>}),
 frozenset({<Gene YDR322C-A at 0x1650b7979d0>}),
 frozenset({<Gene YNL229C at 0x1650b7c9a10>}),
 frozenset({<Gene YCL035C at 0x1650b7a93d0>}),
 frozenset({<Gene YKL211C at 0x1650b821790>}),
 frozenset({<Gene YDR321W at 0x1650b797710>}),
 frozenset({<Gene YLR047C at 0x1650b827190>}),
 frozenset({<Gene YFL001W at 0x1650b7ccd90>}),
 frozenset({<Gene YOL143C at 0x1650b817e50>}),
 frozenset({<Gene YKR076W at 0x1650b7af350>}),
 frozenset({<Gene YBR153W at 0x1650b80c4d0>}),
 frozenset({<Gene YOR120W at 0x1650b82ce90>}),
 frozenset({<Gene YPL271W at 0x1650b7f5dd0>}),
 frozenset({<Gene YML004C at 0x1650b82b7d0>}),
 frozenset({<Gene YIL066C at 0x1650b7de310>}),
 frozenset(

In [37]:
kd_genes_final = list(set(kd_genes))
kd_genes_final

[frozenset({<Gene YBR127C at 0x1650b80e2d0>,
            <Gene YDL185W at 0x1650b794310>,
            <Gene YEL027W at 0x1650b7f9e50>,
            <Gene YEL051W at 0x1650b7fa650>,
            <Gene YGR020C at 0x1650b7d40d0>,
            <Gene YHR026W at 0x1650b7d7ed0>,
            <Gene YHR039C-A at 0x1650b7dc2d0>,
            <Gene YKL080W at 0x1650b820290>,
            <Gene YLR447C at 0x1650b82b5d0>,
            <Gene YMR054W at 0x1650b7e1210>,
            <Gene YOR332W at 0x1650b82ef90>,
            <Gene YPL234C at 0x1650b7f56d0>,
            <Gene YPR036W at 0x1650b7f68d0>}),
 frozenset({<Gene YMR020W at 0x1650b7e1010>}),
 frozenset({<Gene YER183C at 0x1650b7ccc90>}),
 frozenset({<Gene YFR025C at 0x1650b7cd890>}),
 frozenset({<Gene YLR410W at 0x1650b82b1d0>}),
 frozenset({<Gene YER005W at 0x1650b7fac50>}),
 frozenset({<Gene YMR300C at 0x1650b7e3610>}),
 frozenset({<Gene YLR299W at 0x1650b829fd0>}),
 frozenset({<Gene YDR321W at 0x1650b797710>}),
 frozenset({<Gene YOL064C at 0x1650

In [38]:
oe_genes_result = pd.DataFrame(data=oe_genes_final)   
kd_genes_result = pd.DataFrame(data=kd_genes_final)  

In [39]:
oe_genes_result.to_csv('./results/application_result2/OE_genes.csv',index=0)
kd_genes_result.to_csv('./results/application_result2/KD_genes.csv',index=0)

In [40]:
# up_reactions_matched = []
# down_reactions_matched = []

# for up_reaction in up_reaction_list:
#     if re.match(r'r_\d{4}$', up_reaction):
#         up_reactions_matched.append(up_reaction)
        
# for down_reaction in down_reaction_list:
#     if re.match(r'r_\d{4}$', down_reaction):
#         down_reactions_matched.append(down_reaction)

# up_reactions = pd.DataFrame(data=up_reactions_matched,columns=['Up'])   
# down_reactions = pd.DataFrame(data=down_reactions_matched,columns=['Down'])              

# up_reactions.to_csv('./results/application_result2/up_reactions.csv',index=0)
# down_reactions.to_csv('./results/application_result2/down_reactions.csv',index=0)            